Bibliography:

* [Stacked Convolutional Auto-Encoders for Hierarchical Feature Extraction](http://people.idsia.ch/~ciresan/data/icann2011.pdf)
    

In [1]:
import torch
import torchvision
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
from torchvision import datasets
from torchvision.utils import save_image

import skimage 
import math
# import io
# import requests
# from PIL import Image

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import sys
import os


In [2]:
import cae

In [3]:
from helpers import *
from helper_modules import *
from multi_res_cae import *

In [4]:
%load_ext autoreload
%autoreload

In [5]:
%aimport helpers, helper_modules, multi_res_cae

In [6]:
%aimport

Modules to reload:
helper_modules helpers multi_res_cae

Modules to skip:



In [7]:
# frcae = MultiFullCAE((640,480))

In [8]:
# mrcae = MultiResCAE([640,480])

In [9]:
# Hyper Parameters
# num_epochs = 5
# batch_size = 100
# learning_rate = 0.001

num_epochs = 100
batch_size = 128
learning_rate = 0.0001

In [10]:
%%time

#%time model = MultiFullCAE(in_img_shape=(32,32), full_image_resize=(24,24)).cuda()
model = MultiResCAE(in_img_shape=[32,32], channels=3, conv_layer_feat=[16, 16, 32],
                 res_px=[[24, 24], [16, 16], [12, 12]], crop_sizes=[[32, 32], [24,24], [12, 12]],
                 # conv_sizes = [(3,5,7), (3,5,7,11), (3,5,7,11)]  # this is too much I think
                 conv_sizes=[[3, 5], [3, 5], [3, 5, 7]]
        ).cuda()

CPU times: user 1.15 s, sys: 309 ms, total: 1.46 s
Wall time: 1.46 s


In [11]:
# model.parameters

In [12]:
%%time
criterion = nn.MSELoss()
#criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

CPU times: user 559 µs, sys: 264 µs, total: 823 µs
Wall time: 826 µs


In [13]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 3, 32, 32)
    return x


In [14]:
%%time

#transformation = monochrome_preprocess(32,32)
transformation = fullimage_preprocess(32,32)
#train_loader, test_loader = get_loaders(batch_size, transformation, dataset=datasets.CocoDetection)
train_loader, test_loader = get_loaders(batch_size, transformation)

Files already downloaded and verified
CPU times: user 716 ms, sys: 172 ms, total: 888 ms
Wall time: 886 ms


In [ ]:
%%time

for epoch in range(num_epochs):
    for i, (img, labels) in enumerate(train_loader):
        img = Variable(img).cuda()
        # ===================forward=====================
#         print("encoding batch of  images")
        output = model(img)
#         print("computing loss")
        loss = criterion(output, img)
        # ===================backward====================
#         print("Backward ")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        in_pic = to_img(img.cpu().data)
        save_image(pic, './mrcae_results/in-64x64_32x32_3-5-7-11-out_image_{}.png'.format(epoch))
        save_image(in_pic, './mrcae_results/in-64x64_32x32_3-5-7-11-in_image_{}.png'.format(epoch))
    if loss.data[0] < 0.21: #arbitrary number because I saw that it works well enough
        break

prev_crop torch.Size([128, 3, 32, 32])
torch.Size([128, 3, 32, 32])
prev_crop torch.Size([128, 3, 32, 32])
torch.Size([128, 3, 24, 24])
prev_crop torch.Size([128, 3, 24, 24])
torch.Size([128, 3, 12, 12])
encoding crop size =  torch.Size([128, 3, 32, 32])
encoding crop size =  torch.Size([128, 3, 24, 24])
encoding crop size =  torch.Size([128, 3, 12, 12])
ranges =  
  0   0  32  32
  3   4  27  28
  3   6  15  18
[torch.IntTensor of size (3,4)]

merging  0 
  3
  4
 27
 28
[torch.IntTensor of size (4,)]
 torch.Size([128, 3, 24, 24]) torch.Size([128, 3, 32, 32]) torch.Size([128, 3, 24, 24])
merging  1 
  3
  6
 15
 18
[torch.IntTensor of size (4,)]
 torch.Size([128, 3, 12, 12]) torch.Size([128, 3, 24, 24]) torch.Size([128, 3, 12, 12])
prev_crop torch.Size([128, 3, 32, 32])
torch.Size([128, 3, 32, 32])
prev_crop torch.Size([128, 3, 32, 32])
torch.Size([128, 3, 24, 24])
prev_crop torch.Size([128, 3, 24, 24])
torch.Size([128, 3, 12, 12])
encoding crop size =  torch.Size([128, 3, 32, 32])
en

In [ ]:
torch.cat?

In [ ]:
#torch.save("fmrcae_in-64x64_32x32_3-5-7-11.pth", model)
#torch.save("mrcae_in-32x32_.pth", model)

## Preliminary Results 

Experiments with the following configurations:


## Pytorch understanding tests